Import the important libraries

In [1]:
import rospy
from ipywidgets import Button, FloatText, VBox, widgets, Output, HTML
from IPython.display import clear_output
from assignment_2_2023.msg import PlanningAction, PlanningGoal, PlanningResult, Goal
from actionlib_msgs.msg import GoalStatus, GoalStatusArray
import actionlib
from nav_msgs.msg import Odometry
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from tf.transformations import quaternion_matrix
import tf
import numpy as np
import threading
import time




Code to set the target or cancel the target

In [2]:
# ROS node ko initialize karo
rospy.init_node('jupyter_ros_interface')
action_client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
Feedback_Goal = [0, 0]
flag = True

x = []
y = []
# Button click event ke liye callback function define karo
def button_click(lol): 
    global Feedback_Goal
    Feedback_Goal[0] += 1
    target_x = float(x_input.value)
    target_y = float(y_input.value)
    x.append(target_x)
    y.append(target_y)

    # Action server ko goal bhejo
    goal = PlanningGoal()
    goal.target_pose.pose.position.x = target_x
    goal.target_pose.pose.position.y = target_y
    
    action_client.send_goal(goal)
    print("Goal sent with target coordinates:", target_x, target_y)
    
def cancel_clicked(cancel):
    action_client.cancel_goal()
    print("Target has been cancelled, set new target")
    x_input.value = 0
    y_input.value = 0

# Ye interface k liye hai, mtlb interface me jo x or y daalenge. Baaki jo process karega wo to function button_click hai. 
x_input = FloatText(description='Target X:')
y_input = FloatText(description='Target Y:')
cancel_button = Button(description='Cancel', tooltip='Click to cancel operation')

# Submit button banao
submit_button = Button(description='Submit Goal')

# Button click event ko callback function se jodo
submit_button.on_click(button_click)
cancel_button.on_click(cancel_clicked)

# Ye target_x or target_y or submit button ko display karne k liye hai
VBox([x_input, y_input, submit_button,cancel_button])

 

Goal sent with target coordinates: 0.0 0.0
Goal sent with target coordinates:No worries, next time
 0.0 4.0
Goal sent with target coordinates: 0.0 0.0
Goal sent with target coordinates: 0.0 No worries, next time
No worries, next time
No worries, next time
No worries, next time
No worries, next time4.0
Goal sent with target coordinates: 0.0 0.0
Goal sent with target coordinates:No worries, next time
No worries, next time
 No worries, next time
No worries, next time
No worries, next time0.0
No worries, next time
 4.0
No worries, next time
No worries, next time
No worries, next time
No worries, next time
Goal sent with target coordinates: 0.0 0.0
Goal sent with target coordinates: 0.0No worries, next time
No worries, next time
No worries, next time
No worries, next time
No worries, next time
No worries, next time
No worries, next time
No worries, next time
No worries, next time
No worries, next time
No worries, next time
No worries, next time
No worries, next time
No worries, next time
 4

All targets that have been set.

In [3]:
# Convert the lists to strings
x_str = '<br>'.join(map(str, x))
y_str = '<br>'.join(map(str, y))

# Format X and Y coordinates
x_html = f"<b>X coordinates:</b><br>{x_str}"
y_html = f"<b>Y coordinates:</b><br>{y_str}"

# Create HTML widgets to display the data
x_html_widget = HTML(value=x_html)
y_html_widget = HTML(value=y_html)

# Display the HTML widgets
display(VBox([x_html_widget, y_html_widget]))





In [21]:
%matplotlib widget
class Visualiser:
 
    def __init__(self):
        self.fig, (self.ax, self.ax_bar) = plt.subplots(1, 2, figsize=(10, 5))
        self.ln, = self.ax.plot([], [], 'ro', label='Position')
        self.goal_ln, = self.ax.plot([], [], 'bx', label='Goal')
        self.bar_ln = None
        self.x_data, self.y_data = [], []
        self.x_goal, self.y_goal = 0, 0
        self.plot_init()

    def plot_init(self):
        self.ax.set_xlim(-20, 20)
        self.ax.set_ylim(-20, 20)
        
        
        self.ax_bar.set_ylim(0, max(Feedback_Goal) + 1)
        self.bar_ln = self.ax_bar.bar(['Not Reached', 'Reached'], Feedback_Goal, color=['red', 'green'])
        
        return self.ln, self.goal_ln, self.bar_ln
        
    def clbk_odom(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)

    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        self.goal_ln.set_data([self.x_goal], [self.y_goal])
        self.update_second_plot()
        
    def update_second_plot(self):
        self.bar_ln[0].set_height(Feedback_Goal[0])
        self.bar_ln[1].set_height(Feedback_Goal[1])
        self.ax_bar.set_ylim(0, max(Feedback_Goal) + 1)
        return self.ln, self.goal_ln, self.bar_ln
        
    def goal_done_cb(self, msg):
        global flag, Feedback_Goal
        for status in msg.status_list:
            if status.status == GoalStatus.SUCCEEDED:
                if flag == True:
                    Feedback_Goal[0] = 0
                    Feedback_Goal[1] = 1
                    flag = False 
                    print("Success")
                    self.update_second_plot()
                    
    
            else:
                Feedback_Goal[0] = 1
                Feedback_Goal[1] = 0
                flag = True
                print("No worries, next time")
                self.update_second_plot()
        return self.ln, self.goal_ln, self.bar_ln
    
vis = Visualiser()
sub_odom = rospy.Subscriber('/odom', Odometry, vis.clbk_odom)
sub2 = rospy.Subscriber('/reaching_goal/status', GoalStatusArray, vis.goal_done_cb)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func = vis.plot_init, interval=50)
plt.show(block=True)

vis.ax.plot(x_input.value, y_input.value,  marker='*', markersize=5)
plt.draw()



 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
print(Feedback_Goal)

[0, 1]


In [6]:
%matplotlib widget
import matplotlib.pyplot as plt
import rospy
from turtlesim.msg import Pose
from matplotlib.animation import FuncAnimation

class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro')
        self.x_data, self.y_data = [], []

    def plot_init(self):
        self.ax.set_xlim(-20, 20)
        self.ax.set_ylim(-20, 20)
        return self.ln,
        
    def odom_callback(self, msg):
        self.y_data.append(msg.y)
        self.x_data.append(msg.x)

    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln,
    
    def clbk_odom(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
 

vis = Visualiser()
plt.show(block=True)

sub_odom = rospy.Subscriber('/odom', Odometry, vis.clbk_odom)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func = vis.plot_init, interval=50)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …